# Datenimport

In [2]:
import pandas as pd

movies = pd.read_csv("movie_data/movies_metadata.csv", low_memory=False)
movie_actors = pd.read_csv("movie_data/credits.csv", low_memory=False)

# Datensätze verbinden

In [4]:
# join actors / crew and movie metadata on index
movie_actors['id'] = movie_actors['id'].apply(str)
merged = movies.set_index("id").join(movie_actors.set_index('id'))


merged.head(10)

,adult,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew
id,,,,,,,,,,,,,,,,,,,,,
100,False,NaN,1350000,"[{'id': 35, 'name': 'Comedy'}, {'id': 80, 'nam...",http://www.universalstudiosentertainment.com/l...,tt0120735,en,"Lock, Stock and Two Smoking Barrels",A card sharp and his unwillingly-enlisted frie...,4.60786,...,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A Disgrace to Criminals Everywhere.,"Lock, Stock and Two Smoking Barrels",False,7.5,1671.0,"[{'cast_id': 12, 'character': 'Tom', 'credit_i...","[{'credit_id': '52fe4217c3a36847f80035c1', 'de..."
10000,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0109747,es,La estrategia del caracol,A group of tenants living in an old house are ...,0.281609,...,116.0,"[{'iso_639_1': 'es', 'name': 'Español'}]",Released,NaN,La estrategia del caracol,False,7.2,9.0,[],"[{'credit_id': '5375293cc3a3681ee90001ad', 'de..."
10001,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 878, 'na...",NaN,tt0096486,en,Young Einstein,Albert Einstein is the son of a Tasmanian appl...,2.562888,...,91.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,E=mc²,Young Einstein,False,4.5,46.0,"[{'cast_id': 17, 'character': 'Albert Einstein...","[{'credit_id': '52fe43039251416c75000035', 'de..."
100010,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...",NaN,tt0032477,en,Flight Command,"A rookie flyer, Ens. Alan Drake, joins the fam...",0.769266,...,116.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"T-H-R-I-L-L AMERICA! Here come The Flying ""HEL...",Flight Command,False,6.0,1.0,"[{'cast_id': 4, 'character': 'Ens. Alan Drake'...","[{'credit_id': '52fe49c7c3a36847f81a5a21', 'de..."
100017,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,tt0488903,de,Verfolgt,Deals with the obsessive relationship between ...,2.964103,...,87.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,NaN,Hounded,False,4.8,7.0,"[{'cast_id': 1, 'character': 'Jan Winkler', 'c...","[{'credit_id': '53f5e242c3a36833f7003a15', 'de..."
10002,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",NaN,tt0091538,en,Mona Lisa,"George has just been released from prison, and...",3.095584,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Mona Lisa,False,6.7,62.0,"[{'cast_id': 1, 'character': 'George', 'credit...","[{'credit_id': '52fe43039251416c7500010d', 'de..."
100024,False,NaN,0,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",NaN,tt1562567,en,Bloodwork,A couple of college students decide to sign up...,1.099911,...,100.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Not Dying Doesn't Mean You're Alive,Bloodwork,False,4.3,13.0,"[{'cast_id': 1002, 'character': 'Greg', 'credi...","[{'credit_id': '52fe49c7c3a36847f81a5b5b', 'de..."
10003,False,"{'id': 86224, 'name': 'The Saint Collection', ...",68000000,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",NaN,tt0120053,en,The Saint,"Ivan Tretiak, Russian Mafia boss who wants to ...",10.97633,...,116.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Never reveal your name. Never turn your back. ...,The Saint,False,5.9,310.0,"[{'cast_id': 13, 'character': 'Simon Templar',...","[{'credit_id': '52fe43039251416c75000187', 'de..."
100032,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,tt0099137,en,The Great Los Angeles Earthquake,After a series of small tremors in Los Angeles...,1.315925,...,180.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"There is no safe harbor, there is no escape......",The Great Los Angeles Earthquake,False,6.8,5.0,"[{'cast_id': 1, 'character': 'Dr. Clare Winslo...","[{'credit_id': '59ac6d9f925141079d033344', 'de..."


# Datenaufbereitung

In [5]:
# remove elements without title or cast
merged = merged[merged["title"].notnull()]
merged = merged[merged["cast"].notnull()]

# transform data
tf_genres = []
# tf_languages = []
# tf_companies = []
tf_cast = []
directors = []
for index, row in merged.iterrows():
    # genres
    tf_genres_row = []
    genres = eval(row["genres"])
    for genre in genres:
        tf_genres_row.append(genre["name"])
    tf_genres.append(tf_genres_row)
    
#     # languages
#     tf_languages_row = []
#     languages = eval(row["spoken_languages"])
#     for lan in languages:
#         tf_languages_row.append(lan["name"])
#     tf_languages.append(tf_languages_row)
    
#     # companies
#     tf_companies_row = []
#     companies = eval(row["production_companies"])
#     for com in companies:
#         tf_companies_row.append(com["name"])
#     tf_companies.append(tf_companies_row)
    
    # cast
    tf_cast_row = []
    casts = eval(row["cast"])
    for cast in casts:
        tf_cast_row.append(cast["name"])
    tf_cast.append(tf_cast_row)
    
    # director
    crew = eval(row["crew"])
    director = ""
    for person in crew:
        if person["job"]=="Director":
            director = person["name"]
            break
    directors.append(director)

            
# add data to table
merged["tf_genres"] = tf_genres
# merged["tf_languages"] = tf_languages
# merged["tf_companies"] = tf_companies
merged["tf_cast"] = tf_cast
merged["director"] = directors
            
# delete not used columns
col_dels = ["adult", "belongs_to_collection", "homepage", "imdb_id", "poster_path", "production_countries",
            "tagline", "video", "original_title", "status", "original_language", "overview", "budget", "genres",
            "spoken_languages", "production_companies", "cast", "crew"]
for col in col_dels:
    if col in merged.columns:
        del merged[col]

# change order of columns
cols = ['title','tf_genres','tf_cast','director','release_date','vote_average','vote_count','revenue','runtime','popularity']
merged = merged[cols]

merged.head(10)

,title,tf_genres,tf_cast,director,release_date,vote_average,vote_count,revenue,runtime,popularity
id,,,,,,,,,,
100,"Lock, Stock and Two Smoking Barrels","[Comedy, Crime]","[Jason Flemyng, Dexter Fletcher, Nick Moran, J...",Guy Ritchie,1998-03-05,7.5,1671.0,3897569.0,105.0,4.60786
10000,La estrategia del caracol,"[Comedy, Drama]",[],Sergio Cabrera,1993-12-25,7.2,9.0,0.0,116.0,0.281609
10001,Young Einstein,"[Comedy, Science Fiction]","[Yahoo Serious, Odile Le Clezio, Peewee Wilson...",Yahoo Serious,1988-12-15,4.5,46.0,0.0,91.0,2.562888
100010,Flight Command,"[Drama, War]","[Robert Taylor, Ruth Hussey, Walter Pidgeon, P...",Frank Borzage,1940-12-27,6.0,1.0,0.0,116.0,0.769266
100017,Hounded,[Drama],"[Kostja Ullmann, Maren Kroymann, Moritz Grove,...",Angelina Maccarone,2006-08-06,4.8,7.0,0.0,87.0,2.964103
10002,Mona Lisa,"[Drama, Crime, Romance]","[Bob Hoskins, Cathy Tyson, Michael Caine, Robb...",Neil Jordan,1986-06-13,6.7,62.0,5794184.0,104.0,3.095584
100024,Bloodwork,"[Horror, Thriller]","[Travis Van Winkle, Tricia Helfer, Eric Robert...",Eric Wostenberg,2012-05-10,4.3,13.0,0.0,100.0,1.099911
10003,The Saint,"[Thriller, Action, Romance, Science Fiction, A...","[Val Kilmer, Elisabeth Shue, Rade Serbedzija, ...",Phillip Noyce,1997-04-03,5.9,310.0,118063304.0,116.0,10.97633
100032,The Great Los Angeles Earthquake,"[Drama, Action]","[Joanna Kerns, Dan Lauria, Bonnie Bartlett, Li...",Larry Elikann,1990-11-11,6.8,5.0,0.0,180.0,1.315925


In [6]:
# import keywords
keywords = pd.read_csv("movie_data/keywords.csv")

# Keywords transformieren
tf_keywords = []

for index, row in keywords.iterrows():
    tf_row_keywords = []
    for key in eval(row["keywords"]):
        tf_row_keywords.append(key["name"])
    tf_keywords.append(tf_row_keywords)

keywords["tf_keywords"] = tf_keywords
del keywords["keywords"]

keywords

,id,tf_keywords
0,862,"[jealousy, toy, boy, friendship, friends, riva..."
1,8844,"[board game, disappearance, based on children'..."
2,15602,"[fishing, best friend, duringcreditsstinger, o..."
3,31357,"[based on novel, interracial relationship, sin..."
4,11862,"[baby, midlife crisis, confidence, aging, daug..."
...,...,...
46414,439050,[tragic love]
46415,111109,"[artist, play, pinoy]"
46416,67758,[]
46417,227506,[]


In [7]:
# merge keywords and merged
keywords['id'] = keywords['id'].apply(str)
merged = merged.join(keywords.set_index('id'))

In [8]:
# namen konvertieren
def clean_names(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    elif isinstance(x, str):
        # director
        return str.lower(x.replace(" ", ""))

features = ['tf_cast', 'tf_keywords', 'director', 'tf_genres']

for feature in features:
    merged[feature] = merged[feature].apply(clean_names)
    
    
merged

,title,tf_genres,tf_cast,director,release_date,vote_average,vote_count,revenue,runtime,popularity,tf_keywords
id,,,,,,,,,,,
100,"Lock, Stock and Two Smoking Barrels","[comedy, crime]","[jasonflemyng, dexterfletcher, nickmoran, jaso...",guyritchie,1998-03-05,7.5,1671.0,3897569.0,105.0,4.60786,"[ambush, alcohol, shotgun, tea, joint, machism..."
10000,La estrategia del caracol,"[comedy, drama]",[],sergiocabrera,1993-12-25,7.2,9.0,0.0,116.0,0.281609,"[roommate, pastor, squatter, anarchist, house,..."
10001,Young Einstein,"[comedy, sciencefiction]","[yahooserious, odileleclezio, peeweewilson, su...",yahooserious,1988-12-15,4.5,46.0,0.0,91.0,2.562888,"[atomicbomb, nobelprize, rock, alberteinstein,..."
100010,Flight Command,"[drama, war]","[roberttaylor, ruthhussey, walterpidgeon, paul...",frankborzage,1940-12-27,6.0,1.0,0.0,116.0,0.769266,"[pilot, navy]"
100017,Hounded,[drama],"[kostjaullmann, marenkroymann, moritzgrove, sı...",angelinamaccarone,2006-08-06,4.8,7.0,0.0,87.0,2.964103,"[fetishism, masochism, submissive, olderwomany..."
...,...,...,...,...,...,...,...,...,...,...,...
99946,Exit Smiling,[comedy],"[beatricelillie, jackpickford, dorislloyd, dew...",samtaylor,1926-11-06,8.5,2.0,0.0,77.0,0.202315,"[femmefatale, backstage, unrequitedlove, train..."
9995,Turn It Up,"[action, crime, drama]","[jasonstatham, vondiecurtis-hall, tamalajones,...",robertadetuyi,2000-09-06,5.0,5.0,0.0,86.0,1.316179,"[singleparent, rapmusic, lossofmother, sociall..."
9997,Gabriel,"[fantasy, horror, action, sciencefiction]","[dwainestevenson, jackcampbell, erikaheynatz, ...",shaneabbess,2007-11-15,5.0,77.0,0.0,109.0,3.840024,"[archangelgabriel, heaven, matteroflifeanddeat..."


In [9]:
# create merged string of all valued attributes (for now this is gonna be only case, director, keywords and genres)
def create_summary(x):
    return ' '.join(x['tf_keywords']) + ' ' + ' '.join(x['tf_cast']) + ' ' + x['director'] + ' ' + ' '.join(x['tf_genres'])

merged["summary"] = merged.apply(create_summary, axis=1)

,title,tf_genres,tf_cast,director,release_date,vote_average,vote_count,revenue,runtime,popularity,tf_keywords,summary
id,,,,,,,,,,,,
100,"Lock, Stock and Two Smoking Barrels","[comedy, crime]","[jasonflemyng, dexterfletcher, nickmoran, jaso...",guyritchie,1998-03-05,7.5,1671.0,3897569.0,105.0,4.60786,"[ambush, alcohol, shotgun, tea, joint, machism...",ambush alcohol shotgun tea joint machismo cock...
10000,La estrategia del caracol,"[comedy, drama]",[],sergiocabrera,1993-12-25,7.2,9.0,0.0,116.0,0.281609,"[roommate, pastor, squatter, anarchist, house,...",roommate pastor squatter anarchist house bogot...
10001,Young Einstein,"[comedy, sciencefiction]","[yahooserious, odileleclezio, peeweewilson, su...",yahooserious,1988-12-15,4.5,46.0,0.0,91.0,2.562888,"[atomicbomb, nobelprize, rock, alberteinstein,...",atomicbomb nobelprize rock alberteinstein mari...
100010,Flight Command,"[drama, war]","[roberttaylor, ruthhussey, walterpidgeon, paul...",frankborzage,1940-12-27,6.0,1.0,0.0,116.0,0.769266,"[pilot, navy]",pilot navy roberttaylor ruthhussey walterpidge...
100017,Hounded,[drama],"[kostjaullmann, marenkroymann, moritzgrove, sı...",angelinamaccarone,2006-08-06,4.8,7.0,0.0,87.0,2.964103,"[fetishism, masochism, submissive, olderwomany...",fetishism masochism submissive olderwomanyoung...
...,...,...,...,...,...,...,...,...,...,...,...,...
100493,The Devil's Carnival,[horror],"[seanpatrickflanery, brianaevigan, billmoseley...",darrenlynnbousman,2012-04-05,5.3,33.0,0.0,55.0,1.60037,"[clown, musical, lucifer, fable, darkcarnival]",clown musical lucifer fable darkcarnival seanp...
10050,Get Over It,"[comedy, romance]","[kirstendunst, benfoster, melissasagemiller, c...",tommyo'haver,2001-03-08,5.5,76.0,0.0,87.0,7.867139,"[theatreplay, theatregroup, highschool, fallin...",theatreplay theatregroup highschool fallinginl...
10051,Avenue Montaigne,"[comedy, drama, romance]","[céciledefrance, albertdupontel, lauramorante,...",danièlethompson,2006-02-15,6.2,27.0,0.0,106.0,2.084571,"[paris, waitress, bistrot, spectacle, womandir...",paris waitress bistrot spectacle womandirector...


In [10]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(merged.head(2000)['summary'])

count_matrix

<2000x27693 sparse matrix of type '<class 'numpy.int64'>'
	with 45117 stored elements in Compressed Sparse Row format>

In [11]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [12]:
merged = merged.reset_index()
indices = pd.Series(merged.index, index=merged['title'])

merged

,id,title,tf_genres,tf_cast,director,release_date,vote_average,vote_count,revenue,runtime,popularity,tf_keywords,summary
0,100,"Lock, Stock and Two Smoking Barrels","[comedy, crime]","[jasonflemyng, dexterfletcher, nickmoran, jaso...",guyritchie,1998-03-05,7.5,1671.0,3897569.0,105.0,4.60786,"[ambush, alcohol, shotgun, tea, joint, machism...",ambush alcohol shotgun tea joint machismo cock...
1,10000,La estrategia del caracol,"[comedy, drama]",[],sergiocabrera,1993-12-25,7.2,9.0,0.0,116.0,0.281609,"[roommate, pastor, squatter, anarchist, house,...",roommate pastor squatter anarchist house bogot...
2,10001,Young Einstein,"[comedy, sciencefiction]","[yahooserious, odileleclezio, peeweewilson, su...",yahooserious,1988-12-15,4.5,46.0,0.0,91.0,2.562888,"[atomicbomb, nobelprize, rock, alberteinstein,...",atomicbomb nobelprize rock alberteinstein mari...
3,100010,Flight Command,"[drama, war]","[roberttaylor, ruthhussey, walterpidgeon, paul...",frankborzage,1940-12-27,6.0,1.0,0.0,116.0,0.769266,"[pilot, navy]",pilot navy roberttaylor ruthhussey walterpidge...
4,100017,Hounded,[drama],"[kostjaullmann, marenkroymann, moritzgrove, sı...",angelinamaccarone,2006-08-06,4.8,7.0,0.0,87.0,2.964103,"[fetishism, masochism, submissive, olderwomany...",fetishism masochism submissive olderwomanyoung...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46619,99946,Exit Smiling,[comedy],"[beatricelillie, jackpickford, dorislloyd, dew...",samtaylor,1926-11-06,8.5,2.0,0.0,77.0,0.202315,"[femmefatale, backstage, unrequitedlove, train...",femmefatale backstage unrequitedlove train ont...
46620,9995,Turn It Up,"[action, crime, drama]","[jasonstatham, vondiecurtis-hall, tamalajones,...",robertadetuyi,2000-09-06,5.0,5.0,0.0,86.0,1.316179,"[singleparent, rapmusic, lossofmother, sociall...",singleparent rapmusic lossofmother sociallydep...
46621,9997,Gabriel,"[fantasy, horror, action, sciencefiction]","[dwainestevenson, jackcampbell, erikaheynatz, ...",shaneabbess,2007-11-15,5.0,77.0,0.0,109.0,3.840024,"[archangelgabriel, heaven, matteroflifeanddeat...",archangelgabriel heaven matteroflifeanddeath h...
46622,99977,Hot Stuff,[comedy],"[domdeluise, suzannepleshette, jerryreed, ossi...",domdeluise,1979-08-10,7.8,6.0,0.0,91.0,0.215778,"[police, receivingofstolengoods, scam, pawnshop]",police receivingofstolengoods scam pawnshop do...


In [13]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return merged['title'].iloc[movie_indices]

In [15]:
get_recommendations('Hounded', cosine_sim)

434                        Ill Gotten Gains
1075                          You Are Not I
1112    Edie & Thea: A Very Long Engagement
339                                Bluebird
720                           Out of Bounds
1688                          Night and Day
56                               Tussenland
319                               Game Over
511                          Found Memories
617                           Grandma Lo-Fi
Name: title, dtype: object